##Подготовка данных

Устанавливаем вики

In [1]:
!pip install wikipedia -q

import wikipedia

wikipedia.set_lang("ru")

  Preparing metadata (setup.py) ... done


Пишем функцию, потому что это проще, чем обработать пять статей

In [2]:
def wiki_dict(titles: list):
    wiki_dict = {}
    for title in titles:
        page = wikipedia.page(title)
        title = page.title
        text = page.content
        wiki_dict[title] = text
    return wiki_dict

Делаем файл

In [3]:
import json

psycho_list = ["Шизофрения", "Диссоциативное расстройство личности",
               "Антидепрессанты", "NMDA-рецептор", "Альтернативный сплайсинг"]

with open ("psycho_wiki.json", "w", encoding="utf-8") as file:
        json.dump(wiki_dict(psycho_list), file, ensure_ascii=False)
        file.write('')

##Работа с ключевыми словами



Здесь я сразу введу переменные (названия списков), которые далее использую, чтобы было понятно, что к чему

In [4]:
key_words_kb = {} #это ключевые слова для кейберта
key_words_yake = {} #это ключевые фразы для yake
key_phrases_kb = {} #это ключевые слова для кейберта
key_phrases_yake = {} #это ключевые фразы для yake

###Извлечение

In [5]:
with open ("psycho_wiki.json", "r", encoding="utf-8") as file:
    text = json.load(file)

####KeyBert

In [6]:
!pip install keybert -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.7 MB/s eta 0:00:00


In [7]:
from keybert import KeyBERT
kw_model = KeyBERT()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
model = KeyBERT('DeepPavlov/rubert-base-cased')

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Этот код просто берет и находит ключевые _слова_ (в задании была именно такая формулировка)

In [9]:
for item in text:
    doc = text[item]
    keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1, 1),
                                  top_n=5)
    print(keywords)
    key_words_kb[item] = keywords

print(key_words_kb)


[('психомоторные', 0.4655), ('шизоаффективное', 0.4412), ('психотические', 0.4157), ('психотическим', 0.4133), ('гиперлипидемия', 0.409)]
[('психотическим', 0.4069), ('психогенной', 0.3944), ('психотической', 0.3863), ('амнезии', 0.3659), ('клинициста', 0.3184)]
[('антидиабетических', 0.4434), ('серотонинергические', 0.4362), ('антихолинергические', 0.4233), ('иммунодепрессантов', 0.4192), ('психомоторные', 0.4177)]
[('протеинкиназу', 0.472), ('аутофосфорилирование', 0.4604), ('глутамата', 0.3809), ('ионотропный', 0.3351), ('nr2d', 0.3275)]
[('сплайсированных', 0.4019), ('сплайсингом', 0.3974), ('сплайсингу', 0.3875), ('сплайсированные', 0.3787), ('сплайсированными', 0.3697)]
{'Шизофрения': [('психомоторные', 0.4655), ('шизоаффективное', 0.4412), ('психотические', 0.4157), ('психотическим', 0.4133), ('гиперлипидемия', 0.409)], 'Диссоциативное расстройство идентичности': [('психотическим', 0.4069), ('психогенной', 0.3944), ('психотической', 0.3863), ('амнезии', 0.3659), ('клинициста', 0

А вот это уже ключевые _фразы_

In [10]:
for item in text:
    doc = text[item]
    keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1, 3),
                                  top_n=5)
    print(keywords)
    key_phrases_kb[item] = keywords

print(key_phrases_kb)


[('ограниченные психотические симптомы', 0.5146), ('встречается шизоидное расстройство', 0.5125), ('психомоторные нарушения', 0.5114), ('расстройствами шизофренического спектра', 0.508), ('критерии шизофрении неопределённы', 0.5064)]
[('между психотическим проявлением', 0.5029), ('расстройстве идентичности могут', 0.4846), ('различие между психотическим', 0.4627), ('простое диссоциативное расстройство', 0.4542), ('наблюдаются явления деперсонализации', 0.4535)]
[('серотонинергические антидепрессанты существуют', 0.5733), ('трициклические антидепрессанты сочетании', 0.5468), ('трициклические антидепрессанты причиной', 0.5455), ('приёме трициклических антидепрессантов', 0.54), ('трициклические антидепрессанты необходимо', 0.5392)]
[('канал активируют протеинкиназу', 0.5799), ('антагонисты nr2b субъединица', 0.5713), ('ионотропный рецептор глутамата', 0.5674), ('субъединица src киназа', 0.5669), ('киназа участвующая сигнальном', 0.5613)]
[('подвергающихся сплайсингу затем', 0.5513), ('отд

критерии шизофрении неопределенны.

####YAKE

In [11]:
!pip install yake -q
from yake import KeywordExtractor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 14.8 MB/s eta 0:00:00


Ключевые _слова_

In [12]:
for item in text:
    doc = text[item]
    custom_kw_extractor = KeywordExtractor(lan='ru', n=1, top=5)
    keywords = custom_kw_extractor.extract_keywords(doc)
    print(keywords)
    key_words_yake[item] = keywords

print(key_words_yake)

[('шизофрении', np.float64(0.0007233803161211978)), ('шизофрения', np.float64(0.002040812590227257)), ('пациентов', np.float64(0.0040670596558274605)), ('симптомы', np.float64(0.004142916314597882)), ('шизофренией', np.float64(0.004165100035589378))]
[('личности', np.float64(0.002957269038377888)), ('идентичности', np.float64(0.004612573541888912)), ('множественной', np.float64(0.005087975299606159)), ('расстройства', np.float64(0.006308112380720812)), ('англ', np.float64(0.01040164345692182))]
[('СИОЗС', np.float64(0.0011473650792233508)), ('ТЦА', np.float64(0.0011882975400943313)), ('антидепрессантов', np.float64(0.0014663589522897057)), ('антидепрессанты', np.float64(0.0021649718588081022)), ('препаратов', np.float64(0.0049154579115894435))]
[('NMDA', np.float64(0.05402821150712233)), ('рецептора', np.float64(0.08341444993790709)), ('NMDA-рецептор', np.float64(0.08524494911371412)), ('субъединиц', np.float64(0.08854006315773773)), ('подтипа', np.float64(0.11624982562416722))]
[('спл

Ключевые фразы + местами слова

In [13]:
for item in text:
    doc = text[item]
    custom_kw_extractor = KeywordExtractor(lan='ru', n=3, top=5)
    keyphrases = custom_kw_extractor.extract_keywords(doc)
    print(keyphrases)
    key_phrases_yake[item] = keyphrases

print(key_phrases_yake)

[('шизофрении', np.float64(0.0007233803161211978)), ('шизофрения', np.float64(0.002040812590227257)), ('пациентов', np.float64(0.0040670596558274605)), ('симптомы', np.float64(0.004142916314597882)), ('шизофренией', np.float64(0.004165100035589378))]
[('расстройство множественной личности', np.float64(0.0012618370988840418)), ('множественной личности', np.float64(0.0012787515030042646)), ('диссоциативное расстройство идентичности', np.float64(0.002239175320719736)), ('личности', np.float64(0.002957269038377888)), ('идентичности', np.float64(0.004612573541888912))]
[('СИОЗС', np.float64(0.0011473650792233508)), ('ТЦА', np.float64(0.0011882975400943313)), ('антидепрессантов группы СИОЗС', np.float64(0.0013596685106345438)), ('антидепрессантов', np.float64(0.0014663589522897057)), ('антидепрессанты', np.float64(0.0021649718588081022))]
[('ионотропный рецептор глутамата', np.float64(0.008729351848245012)), ('селективно связывающий', np.float64(0.02089590337467781)), ('Структурно NMDA-рецеп

###Предварительная лемматизация

In [14]:
!pip install pymorphy3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 69.9 MB/s eta 0:00:00


In [15]:
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

Лемматизация с помощью майстема

In [16]:
def lemmatizer(word):

    analysis = morph.parse(word)[0]
    lemma = analysis.normal_form
    pos = analysis.tag.POS
    if pos == 'PRTF' or pos == 'INFN':
    #это у него были какие-то заскоки, пришлось убирать руками
        pos = 'VERB'
    word_to_vector = f'{lemma}_{pos}'
    return word_to_vector

In [17]:
key_words_kb_lemmas = []
key_words_yake_lemmas = []
key_phrases_kb_lemmas = []
key_phrases_yake_lemmas = []

*Это тоже можно было сделать с помощью функции, но проще (не помню, почему) было так.*

Но вообще этот код просто лемматизирует все ключевые слова везде.

*Дальше у меня, видимо, чуть-чуть неправильная тактика, потому что почти все списки ключевых слов - это именно общий словарь, без разделения на блоки по статьям. Но мне, кажется, особо это не помешало.*

In [18]:
for keywords in key_words_kb.values():
    kw_list = []
    for keyword in keywords:
        kw_list.append(lemmatizer(keyword[0]))
    key_words_kb_lemmas.append(kw_list)

for keywords in key_words_yake.values():
    kw_list = []
    for keyword in keywords:
        kw_list.append(lemmatizer(keyword[0]))
    key_words_yake_lemmas.append(kw_list)

for keywords in key_phrases_yake.values():
    kw_list = []
    for keyword in keywords:
        kph_list = []
        for word in keyword[0].split(' '):
            kph_list.append(lemmatizer(word))
        kw_list.append(kph_list)
    key_phrases_yake_lemmas.append(kw_list)

for keywords in key_phrases_kb.values():
    kw_list = []
    for keyword in keywords:
        kph_list = []
        for word in keyword[0].split(' '):
            kph_list.append(lemmatizer(word))
        kw_list.append(kph_list)
    key_phrases_kb_lemmas.append(kw_list)


In [19]:
print(key_words_kb_lemmas)

[['психомоторный_ADJF', 'шизоаффективный_ADJF', 'психотический_ADJF', 'психотический_ADJF', 'гиперлипидемия_NOUN'], ['психотический_ADJF', 'психогенный_ADJF', 'психотический_ADJF', 'амнезия_NOUN', 'клиницист_NOUN'], ['антидиабетический_ADJF', 'серотонинергический_ADJF', 'антихолинергический_ADJF', 'иммунодепрессант_NOUN', 'психомоторный_ADJF'], ['протеинкиназа_NOUN', 'аутофосфорилирование_NOUN', 'глутамат_NOUN', 'ионотропный_ADJF', 'nr2d_None'], ['сплайсировать_VERB', 'сплайсинг_NOUN', 'сплайсинг_NOUN', 'сплайсировать_VERB', 'сплайсированный_ADJF']]


In [20]:
print(key_phrases_kb_lemmas)

[[['ограниченный_ADJF', 'психотический_ADJF', 'симптом_NOUN'], ['встречаться_VERB', 'шизоидный_ADJF', 'расстройство_NOUN'], ['психомоторный_ADJF', 'нарушение_NOUN'], ['расстройство_NOUN', 'шизофренический_ADJF', 'спектр_NOUN'], ['критерий_NOUN', 'шизофрения_NOUN', 'неопределённый_ADJS']], [['между_PREP', 'психотический_ADJF', 'проявление_NOUN'], ['расстройство_NOUN', 'идентичность_NOUN', 'мочь_VERB'], ['различие_NOUN', 'между_PREP', 'психотический_ADJF'], ['простой_ADJF', 'диссоциативный_ADJF', 'расстройство_NOUN'], ['наблюдаться_VERB', 'явление_NOUN', 'деперсонализация_NOUN']], [['серотонинергический_ADJF', 'антидепрессант_NOUN', 'существовать_VERB'], ['трициклический_ADJF', 'антидепрессант_NOUN', 'сочетание_NOUN'], ['трициклический_ADJF', 'антидепрессант_NOUN', 'причина_NOUN'], ['приём_NOUN', 'трициклический_ADJF', 'антидепрессант_NOUN'], ['трициклический_ADJF', 'антидепрессант_NOUN', 'необходимо_PRED']], [['канал_NOUN', 'активировать_VERB', 'протеинкиназа_NOUN'], ['антагонист_NOUN',

In [21]:
print(key_words_yake_lemmas)

[['шизофрения_NOUN', 'шизофрения_NOUN', 'пациент_NOUN', 'симптом_NOUN', 'шизофрения_NOUN'], ['личность_NOUN', 'идентичность_NOUN', 'множественный_ADJF', 'расстройство_NOUN', 'англ_ADJF'], ['сиозс_NOUN', 'тца_None', 'антидепрессант_NOUN', 'антидепрессант_NOUN', 'препарат_NOUN'], ['nmda_None', 'рецептор_NOUN', 'nmda-рецептор_NOUN', 'субъединица_NOUN', 'подтип_NOUN'], ['сплайсинг_NOUN', 'мрнк_NOUN', 'альтернативный_ADJF', 'сплайсинг_NOUN', 'экзон_NOUN']]


In [22]:
print(key_phrases_yake_lemmas)

[[['шизофрения_NOUN'], ['шизофрения_NOUN'], ['пациент_NOUN'], ['симптом_NOUN'], ['шизофрения_NOUN']], [['расстройство_NOUN', 'множественный_ADJF', 'личность_NOUN'], ['множественный_ADJF', 'личность_NOUN'], ['диссоциативный_ADJF', 'расстройство_NOUN', 'идентичность_NOUN'], ['личность_NOUN'], ['идентичность_NOUN']], [['сиозс_NOUN'], ['тца_None'], ['антидепрессант_NOUN', 'группа_NOUN', 'сиозс_NOUN'], ['антидепрессант_NOUN'], ['антидепрессант_NOUN']], [['ионотропный_ADJF', 'рецептор_NOUN', 'глутамат_NOUN'], ['селективный_ADJS', 'связывать_VERB'], ['структурно_ADVB', 'nmda-рецептор_NOUN', 'представлять_VERB'], ['nmda_None'], ['ионотропный_ADJF', 'рецептор_NOUN']], [['сплайсинг_NOUN'], ['альтернативный_ADJF', 'сплайсинг_NOUN'], ['мрнк_NOUN'], ['альтернативный_ADJF'], ['альтернативный_ADJF', 'сплайсинг_NOUN']]]


###Векторизация

In [23]:
!pip install gensim -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 20.3 MB/s eta 0:00:00


In [24]:
import gensim
import urllib.request

In [25]:
from urllib.request import urlretrieve

In [26]:
urllib.request.urlretrieve('http://vectors.nlpl.eu/repository/20/180.zip',
                           'ruscorpora_upos_cbow_300_20_2019.zip')




('ruscorpora_upos_cbow_300_20_2019.zip',
 <http.client.HTTPMessage at 0x7987f32e4710>)

In [27]:
import zipfile

src = 'ruscorpora_upos_cbow_300_20_2019.zip'

with zipfile.ZipFile(src, 'r') as zip_ref:
    zip_ref.extractall('.')
m = 'model.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

In [28]:
import numpy as np

In [29]:
def vectorizer(list_):
    mean_vectors = {}
    for list_ in list_:
        for phrase in list_:
            all_vectors = []
            if type(phrase) == str:
                if phrase in model:
                    vectors = model[phrase][:10]
                    all_vectors.append(vectors)
                else:
                    pass
                if all_vectors:
                    mean_vectors[''.join(phrase)] = all_vectors
            else:
                for word in phrase:
                    if word in model:
                        vectors = model[word][:10]
                        all_vectors.append(vectors)
                    else:
                        pass
                      #print(f'Увы, слова {word} нет в модели!')
                if all_vectors:
                    vectors_array = np.array(all_vectors)
                    mean_vector = np.mean(vectors_array, axis=0)
                    mean_vectors[' '.join(phrase)] = mean_vector
    return mean_vectors

In [30]:
print(vectorizer(key_words_kb_lemmas))

{'амнезия_NOUN': [array([-0.36807543,  0.21746558,  1.2127441 ,  0.1142756 ,  0.29510128,
        0.5573649 ,  0.07784929, -0.51940316, -1.5812863 ,  0.04195474],
      dtype=float32)], 'клиницист_NOUN': [array([ 0.10410075, -1.1875925 , -0.03729125,  0.8017794 ,  0.22532904,
        0.571649  , -0.81500095,  0.34336016,  0.53905   ,  0.73658705],
      dtype=float32)], 'сплайсинг_NOUN': [array([ 0.98644584,  0.36490446, -0.04065929, -0.37155625,  0.00151187,
       -0.16807833,  0.06322397, -0.18489563,  0.17645459, -0.1361351 ],
      dtype=float32)]}


Это вот все, что он смог достать

In [31]:
print(vectorizer(key_phrases_kb_lemmas))

{'ограниченный_ADJF психотический_ADJF симптом_NOUN': array([ 1.9979749 , -0.15865842,  1.080855  ,  2.3575752 , -2.047815  ,
        2.2376347 , -0.02253645,  0.2360142 , -3.848878  ,  0.76529145],
      dtype=float32), 'встречаться_VERB шизоидный_ADJF расстройство_NOUN': array([-1.7057555 , -0.83858067, -0.0277634 ,  0.98679495, -2.8165264 ,
        1.6869695 ,  0.8940407 , -0.08574314, -2.5914855 , -0.94323885],
      dtype=float32), 'психомоторный_ADJF нарушение_NOUN': array([ 1.350835  , -0.23698917,  2.4674764 ,  1.726857  , -0.29464415,
       -0.1669804 ,  0.04598203, -0.4857541 , -3.5530157 , -1.2300472 ],
      dtype=float32), 'расстройство_NOUN шизофренический_ADJF спектр_NOUN': array([ 1.1756092 , -3.6319203 ,  0.88878214,  0.81888264, -0.7603427 ,
        0.25784862,  0.03669202, -0.20043615, -2.0980012 , -1.2124517 ],
      dtype=float32), 'критерий_NOUN шизофрения_NOUN неопределённый_ADJS': array([ 1.6672059 ,  0.4375276 ,  0.61934155,  0.5261281 , -0.06383574,
        0

Здесь и далее результаты лучше

In [32]:
print(vectorizer(key_words_yake_lemmas))

{'шизофрения_NOUN': [array([ 0.45026803, -1.5297817 ,  0.49584636,  0.23934054, -1.2901037 ,
        1.7760535 , -0.21392848,  0.830179  , -3.8851957 , -0.9808593 ],
      dtype=float32)], 'пациент_NOUN': [array([ 1.2757802 ,  1.2317204 ,  0.4905457 ,  3.31824   ,  0.78440183,
       -0.5422858 , -3.3122957 , -1.465235  , -1.7692944 ,  0.7510204 ],
      dtype=float32)], 'симптом_NOUN': [array([ 1.9979749 , -0.15865842,  1.080855  ,  2.3575752 , -2.047815  ,
        2.2376347 , -0.02253645,  0.2360142 , -3.848878  ,  0.76529145],
      dtype=float32)], 'личность_NOUN': [array([ 0.05183442, -1.4448429 ,  1.89382   , -2.664598  ,  3.4249635 ,
        2.1303592 , -3.1813588 ,  0.30420744, -2.080261  , -2.239991  ],
      dtype=float32)], 'идентичность_NOUN': [array([-0.29161033,  1.352311  ,  0.2994045 , -1.2232623 ,  3.6461232 ,
       -2.1247041 ,  0.19278654, -2.7007773 , -1.7209578 , -0.79334366],
      dtype=float32)], 'расстройство_NOUN': [array([ 0.6620941 , -3.1567247 ,  0.792735 

In [33]:
print(vectorizer(key_phrases_yake_lemmas))

{'шизофрения_NOUN': array([ 0.45026803, -1.5297817 ,  0.49584636,  0.23934054, -1.2901037 ,
        1.7760535 , -0.21392848,  0.830179  , -3.8851957 , -0.9808593 ],
      dtype=float32), 'пациент_NOUN': array([ 1.2757802 ,  1.2317204 ,  0.4905457 ,  3.31824   ,  0.78440183,
       -0.5422858 , -3.3122957 , -1.465235  , -1.7692944 ,  0.7510204 ],
      dtype=float32), 'симптом_NOUN': array([ 1.9979749 , -0.15865842,  1.080855  ,  2.3575752 , -2.047815  ,
        2.2376347 , -0.02253645,  0.2360142 , -3.848878  ,  0.76529145],
      dtype=float32), 'расстройство_NOUN множественный_ADJF личность_NOUN': array([ 0.35696426, -2.3007839 ,  1.3432775 ,  0.4178226 ,  0.39396298,
        2.5339298 , -0.66987985, -0.00447646, -2.240038  , -1.7217029 ],
      dtype=float32), 'множественный_ADJF личность_NOUN': array([ 0.05183442, -1.4448429 ,  1.89382   , -2.664598  ,  3.4249635 ,
        2.1303592 , -3.1813588 ,  0.30420744, -2.080261  , -2.239991  ],
      dtype=float32), 'диссоциативный_ADJF ра

Мне понравились результаты! Чтоб я так жила: в модели есть слова "антидепрессант", "идентичность" и "деперсонализация", но нет "между", "затем" и "простой". Интересно люди живут.

---



##Работа с ИГ

In [34]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 32.2 MB/s  0:00:01
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.8
    Uninstalling spacy-3.8.8:
      Successfully uninstalled spacy-3.8.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.0 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [35]:
import spacy

nlp = spacy.load("ru_core_news_sm")

Эта функция ищет именные группы - мне кажется, что она находит на все (в частности, кажется, не все правильно в перечислении тегов dep_), но в целом работает неплохо.

In [36]:
def noun_group_finder(text):
    d = nlp(text)
    noun_groups = []

    for token in d:
        if token.pos_ == 'NOUN':
            group_draft = []
            group_draft.append(token)

            for child in token.children:
                if child.dep_ in ['amod', 'compound', 'nmod']:
                    group_draft.append(child)

            if token.head.pos_ == 'NOUN' and token.dep_ in ['nmod', 'amod'
                                                            'compound']:
                group_draft.append(token.head)

            if len(group_draft) > 1:
                group_draft = sorted(group_draft, key=lambda x: len(x))
                noun_groups.append(' '.join(
                    [word.text for word in group_draft]))

    return noun_groups

In [37]:
noun_groups_dict = {}
for item in text:
    groups = noun_group_finder(text[item])
    noun_groups_dict[item] = groups

print(noun_groups_dict['Шизофрения'][10:25]) #пример вывода

['болезни проявлениями', 'слуховые псевдогаллюцинации', 'ложные воспоминания', 'бред параноидный', 'речи дезорганизованность', 'речи дезорганизованность', 'фоне дисфункции', 'фоне дисфункции социальной значительной', 'нарушения работоспособности', 'нарушения работоспособности', 'том дебаты', 'единым заболеванием', 'ряд синдромов', 'ряд синдромов отдельных', 'выборе названия']


Инверсия мне тоже не нравится - но я не вспомнила за час до ддл, как работать с сортировкой - может быть, потом попробую поправить.

upd: вспомнила, поправила, но почему-то оно работает не для всего (не могу найти ошибку). Но в свое оправдание скажу, что едва ли это влияет на подсчет среднего вектора, ахах

In [38]:
noun_group_dict_vec = {}
for item in noun_groups_dict:
    lemmas_vec = []
    for noun_group in noun_groups_dict[item]:
        lemmas = []
        for word in noun_group.split():
            word = lemmatizer(word)
            lemmas.append(word)
        lemmas_vec.append(lemmas)
    noun_group_dict_vec[item] = vectorizer([lemmas_vec])
    #я переделала формат вывода и теперь оно хорошо работает!
    print(lemmas_vec)

print(noun_group_dict_vec['Шизофрения']['полиморфный_ADJF психический_ADJF расстройство_NOUN']) #один из блоков словаря - пример вывода



[['полиморфный_ADJF', 'психический_ADJF', 'расстройство_NOUN'], ['распад_NOUN', 'процесс_NOUN'], ['мышление_NOUN', 'распад_NOUN', 'процесс_NOUN'], ['мышление_NOUN', 'процесс_NOUN'], ['реакция_NOUN', 'эмоциональный_ADJF'], ['расстройство_NOUN', 'шизофренический_ADJF'], ['мышление_NOUN', 'характерный_ADJF', 'расстройство_NOUN', 'фундаментальный_ADJF'], ['мышление_NOUN', 'расстройство_NOUN'], ['аффект_NOUN', 'неадекватный_ADJF'], ['частый_ADJF', 'болезнь_NOUN', 'проявление_NOUN'], ['болезнь_NOUN', 'проявление_NOUN'], ['слуховой_ADJF', 'псевдогаллюцинация_NOUN'], ['ложный_ADJF', 'воспоминание_NOUN'], ['бред_NOUN', 'параноидный_ADJF'], ['речь_NOUN', 'дезорганизованность_NOUN'], ['речь_NOUN', 'дезорганизованность_NOUN'], ['фон_NOUN', 'дисфункция_NOUN'], ['фон_NOUN', 'дисфункция_NOUN', 'социальный_ADJF', 'значительный_ADJF'], ['нарушение_NOUN', 'работоспособность_NOUN'], ['нарушение_NOUN', 'работоспособность_NOUN'], ['тот_ADJF', 'дебаты_NOUN'], ['единый_ADJF', 'заболевание_NOUN'], ['ряд_NOUN'

##Проверка близости

###Текстовые совпадения

Здесь я буду работать только с ключевыми фразами, без слов уже.

In [39]:
def checker(list_lemmas):
    lemmas_final = set()
    for lemmas in list_lemmas:
        for lemma in lemmas:
            for word in lemma:
                lemmas_final.add(word)
    return lemmas_final

In [40]:
checker_ng = set()
for item in noun_group_dict_vec:
    for item in noun_group_dict_vec[item]:
        for word in item.split():
            checker_ng.add(word)

In [41]:
checker_kb = checker(key_phrases_kb_lemmas)
checker_yake = checker(key_phrases_yake_lemmas)

In [42]:
all_checker = set(list(checker_kb) + list(checker_yake) + list(checker_ng))

In [43]:
print(checker_kb, '\n', checker_yake, '\n', checker_ng)

{'неопределённый_ADJS', 'серотонинергический_ADJF', 'экзон_NOUN', 'различие_NOUN', 'nr2b_None', 'рецептор_NOUN', 'сочетание_NOUN', 'шизофрения_NOUN', 'отдельный_ADJF', 'протеинкиназа_NOUN', 'психомоторный_ADJF', 'психотический_ADJF', 'наблюдаться_VERB', 'шизоидный_ADJF', 'между_PREP', 'встречаться_VERB', 'являться_VERB', 'подвергаться_VERB', 'ионотропный_ADJF', 'активировать_VERB', 'сайленсер_NOUN', 'src_None', 'явление_NOUN', 'канал_NOUN', 'глутамат_NOUN', 'участвовать_VERB', 'расстройство_NOUN', 'симптом_NOUN', 'это_PRCL', 'затем_ADVB', 'диссоциативный_ADJF', 'антагонист_NOUN', 'спектр_NOUN', 'причина_NOUN', 'киназ_NOUN', 'существовать_VERB', 'приём_NOUN', 'транскрипта_NOUN', 'идентичность_NOUN', 'простой_ADJF', 'трициклический_ADJF', 'сигнальный_ADJF', 'антидепрессант_NOUN', 'необходимо_PRED', 'критерий_NOUN', 'мочь_VERB', 'при_PREP', 'деперсонализация_NOUN', 'репрессор_NOUN', 'сплайсинг_NOUN', 'проявление_NOUN', 'субъединица_NOUN', 'нарушение_NOUN', 'ограниченный_ADJF', 'шизофренич

In [44]:
words_everywhere = []
words_unique = []

for word in all_checker:
    if word in checker_kb and word in checker_yake and word in checker_ng:
        words_everywhere.append(word)
    else:
        words_unique.append(word)

In [45]:
print(words_everywhere)
print(words_unique)

['шизофрения_NOUN', 'идентичность_NOUN', 'сплайсинг_NOUN', 'глутамат_NOUN', 'симптом_NOUN', 'ионотропный_ADJF', 'антидепрессант_NOUN', 'расстройство_NOUN', 'диссоциативный_ADJF', 'рецептор_NOUN']
['тираминовый_ADJF', 'удовлетворение_NOUN', 'старый_ADJF', 'повторный_ADJF', 'прокаинамид_NOUN', 'фермент_NOUN', 'антацида_NOUN', 'генерализовать_VERB', 'взгляд_NOUN', 'индивидуальный_ADJF', 'городской_ADJF', 'ответственность_NOUN', 'затруднить_VERB', 'швеция_NOUN', 'семинар_NOUN', 'фактор_NOUN', 'появление_NOUN', 'книга_NOUN', 'толчкообразный_ADJF', 'резкий_ADJF', 'стресс_NOUN', 'вяло_ADVB', '11-летний_ADJF', 'развлекательный_ADJF', 'обученный_ADJF', 'местный_ADJF', 'надёжность_NOUN', 'льюин_NOUN', 'угроза_NOUN', 'западный_ADJF', 'канва_NOUN', 'гепатотоксический_ADJF', 'обстоятельство_NOUN', 'мг_NOUN', 'приказ_NOUN', 'информация_NOUN', 'паранойяльный_ADJF', 'адренергический_ADJF', 'генерация_NOUN', 'лихорадка_NOUN', 'адаптировать_VERB', 'передача_NOUN', 'немедленный_ADJF', 'переживать_VERB', 

Я случайно сравнила для всех статей, но даже на примере этого блока можно увидеть, что слов, которые выделяет и выделитель именных групп, и оба векторизатора, достаточно мало: это самые-самые ключевые слова.

Насколько я могу поверхностно судить по тому, какие именно слова выделил каждый из вариантов анализа, YAKE выделяет сильно меньше разных неспецифичных слов типа "явление" - наверное, он пытается считать тематику текста и выделить только действительно ключевые, а не только часто встречающиеся.  

###Семантическая близость

In [46]:
print(noun_group_dict_vec['Шизофрения']['полиморфный_ADJF психический_ADJF расстройство_NOUN'])

[ 0.6620941  -3.1567247   0.792735    3.5002432  -2.6370375   2.9375002
  1.8415991  -0.31316036 -2.3998146  -1.2034149 ]


In [47]:
def get_cos_sim(a, b):
    return a.dot(b) / (np.linalg.norm(a) * np.linalg.norm(b))

#код взят с системного блока - это подсчет косинусной близости через нампай

In [48]:
similarity_checker = []
for item in noun_group_dict_vec['Альтернативный сплайсинг']:
    vector = noun_group_dict_vec['Альтернативный сплайсинг'][item]
    for item_compare in noun_group_dict_vec['NMDA-рецептор']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['NMDA-рецептор'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        if max(vector_checker) < 1.0:
        #мне хочется не учитывать одинаковые слова,чтобы не искажать результаты
            similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.2253701


In [49]:
#то же самое с учетом одинаковых слов:

similarity_checker = []
for item in noun_group_dict_vec['Альтернативный сплайсинг']:
    vector = noun_group_dict_vec['Альтернативный сплайсинг'][item]
    for item_compare in noun_group_dict_vec['NMDA-рецептор']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['NMDA-рецептор'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        #мне хочется не учитывать одинаковые слова,чтобы не искажать результаты
        similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.22559363


Здесь приведены два варианта кода - с ограничением на совпадающие слова и без него (второй вариант - без): прямо скажем, это влияет не очень сильно. Но, справедливости ради, это статьи прямо из совсем-совсем разных областей биологии - генетики и биохимии соответственно; в таких частных вещах они не то чтобы сильно пересекаются - поэтому индекс показывает, кажется, не очень высокий уровень семантической близости. И там, и там упоминаются разные биологические термины - этого, вероятно, достаточно.
Мне кажется, что в общем случае точнее использование кода без ограничения.

Пробуем для остальных статей.


In [50]:
similarity_checker = []
for item in noun_group_dict_vec['Шизофрения']:
    vector = noun_group_dict_vec['Шизофрения'][item]
    for item_compare in noun_group_dict_vec['Антидепрессанты']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['Антидепрессанты'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.21428235


In [51]:
similarity_checker = []
for item in noun_group_dict_vec['Антидепрессанты']:
    vector = noun_group_dict_vec['Антидепрессанты'][item]
    for item_compare in noun_group_dict_vec['NMDA-рецептор']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['NMDA-рецептор'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.20562002


In [52]:
similarity_checker = []
for item in noun_group_dict_vec['Шизофрения']:
    vector = noun_group_dict_vec['Шизофрения'][item]
    for item_compare in noun_group_dict_vec['NMDA-рецептор']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['NMDA-рецептор'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.17846236


Ожидаемо самый низкий уровень сходства – у вообще не связанных между собой явлений. Никакого отношения шизофрения не имеет к этому рецептору - это отлично демонстрирует индекс.

In [53]:
similarity_checker = []
for item in noun_group_dict_vec['Антидепрессанты']:
    vector = noun_group_dict_vec['Антидепрессанты'][item]
    for item_compare in noun_group_dict_vec['Альтернативный сплайсинг']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['Альтернативный сплайсинг'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.18424769


In [54]:
similarity_checker = []
for item in noun_group_dict_vec['Шизофрения']:
    vector = noun_group_dict_vec['Шизофрения'][item]
    for item_compare in noun_group_dict_vec['Альтернативный сплайсинг']:
        vector_checker = []
        vector_compare = noun_group_dict_vec['Альтернативный сплайсинг'][item_compare]
        similarity = get_cos_sim(vector, vector_compare)
        vector_checker.append(similarity)
        similarity_checker.append(max(vector_checker))

print(np.mean(similarity_checker))

0.15675476


Извините, это был не самый низкий. Ну, здесь связи еще меньше - психиатрия и генетика вообще никак не связаны между собой (так что тоже ожидаемо).

*Ответ на вопрос "зачем это нужно" для меня, если честно, не очевиден в контексте моего случая (я знаю о содержании этих статей и примерно могу понять, в чем разница) и в контексте различия современных текстов с понятной тематикой в целом (особенно - научных). Но вот, например, для сравнительного анализа текстов разных эпох или для каких-то методик, сопровождающих стилометрию, когда мы анализируем тексты разных авторов и/или разных стилей. Или в контексте лингвистики это может помогать анализировать схожие языки и проводить границу между языком и диалектом.*

##Задание на 9-10

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
corpus = []
for item in text:
    corpus.append(text[item])

print(corpus)

["Шизофрени́я (от др.-греч. σχίζω «расщеплять», «раскалывать» + φρήν «ум, мышление, мысль») — полиморфное психическое расстройство, характеризующееся распадом процессов мышления и эмоциональных реакций.\nШизофренические расстройства, в целом, отличаются характерными фундаментальными расстройствами мышления и восприятия, а также неадекватным или сниженным аффектом. Наиболее частыми проявлениями болезни являются слуховые псевдогаллюцинации, конфабуляции (ложные воспоминания), параноидный или фантастический бред либо дезорганизованность речи и мышления на фоне значительной социальной дисфункции и нарушения работоспособности.\nМногообразие симптоматики породило дебаты о том, является ли шизофрения единым заболеванием или представляет собой диагноз, за которым кроется ряд отдельных синдромов. Эта неоднозначность была отражена при выборе названия: Блейлер использовал множественное число, именуя болезнь шизофрениями.\nЭтимология слова, от «расщепления рассудка», вызывает путаницу — в популярн

In [60]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()

простите я НЕ ПОНЯЛА откуда взялись китайские иероглифы.

In [61]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12370 stored elements and shape (5, 9669)>
  Coords	Values
  (0, 9392)	0.003916469483669385
  (0, 5319)	0.06531765427497062
  (0, 2608)	0.03147486309495198
  (0, 2169)	0.0063195658386546155
  (0, 799)	0.003916469483669385
  (0, 7126)	0.003916469483669385
  (0, 7046)	0.003916469483669385
  (0, 801)	0.003916469483669385
  (0, 8847)	0.003916469483669385
  (0, 4267)	0.00783293896733877
  (0, 4258)	0.0031597829193273077
  (0, 5933)	0.003916469483669385
  (0, 6778)	0.0031597829193273077
  (0, 7098)	0.03672067361077732
  (0, 9181)	0.00783293896733877
  (0, 7050)	0.0031597829193273077
  (0, 6711)	0.0031597829193273077
  (0, 4269)	0.039164694836693846
  (0, 9515)	0.011749408451008154
  (0, 7142)	0.012639131677309231
  (0, 9397)	0.003916469483669385
  (0, 7093)	0.07081844196364197
  (0, 9254)	0.018958697515963848
  (0, 5374)	0.0031597829193273077
  (0, 9191)	0.0031597829193273077
  :	:
  (4, 7253)	0.0066829876077133645
  (4, 6990)	0.0

Код для ключевых *слов*, не фраз:

In [65]:
key_words_tf = []

for i in range(X.shape[0]):
    row = X[i].toarray().flatten()
    kw_tf_sorted = np.argsort(row)[-5:][::-1]
    kw_tf = [(feature_names[idx], row[idx]) for idx in kw_tf_sorted if row[idx] > 0]
    key_words_tf.append(kw_tf)

In [66]:
for i, words in enumerate(key_words_tf):
    print([word for word, score in words])

['шизофрении', 'шизофрения', 'на', 'что', 'не']
['личности', 'множественной', 'идентичности', 'расстройства', 'как']
['антидепрессантов', 'при', 'сиозс', 'тца', 'антидепрессанты']
['рецептор', 'nmda', 'nr2b', 'рецептора', 'субъединиц']
['сплайсинга', 'мрнк', 'альтернативного', 'экзона', 'сплайсинг']


Это очень похоже на правду и, на мой взгляд, это гораздо точнее, чем то, что выделяли другие программы - как мы видели по спискам ключевых слов KeyBERT'a и YAKE, модель часто вообще не может токенизировать определенные слова и в итоге токенизирует не то; а в именные группы просто идет, грубо говоря, все что можно. При этом (как видно по шизофреническому списку) TF-Idf учитывает условные стоп-слова при составлении списка, что в остальных списках при этом не мешает (то есть, на маленьких специализированных текстах, особенно если предварительно они были почищены, это супер эффективный способ). Но, допустим, с какими-нибудь художественными текстами мне не кажется, что это эффективнее, чем KeyBERT.